### Load Data

In [ ]:
# !pip install opencv-python-headless transformers torch torchvision pillow scikit-learn


   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
    --------------------------------------- 0.8/39.4 MB 24.1 MB/s eta 0:00:02
   - -------------------------------------- 1.8/39.4 MB 18.6 MB/s eta 0:00:03
   -- ------------------------------------- 2.8/39.4 MB 19.9 MB/s eta 0:00:02
   ---- ----------------------------------- 4.0/39.4 MB 21.6 MB/s eta 0:00:02
   ----- ---------------------------------- 5.2/39.4 MB 22.4 MB/s eta 0:00:02
   ------ --------------------------------- 6.3/39.4 MB 21.2 MB/s eta 0:00:02
   ------- -------------------------------- 7.0/39.4 MB 20.3 MB/s eta 0:00:02
   -------- ------------------------------- 8.0/39.4 MB 21.4 MB/s eta 0:00:02
   --------- ------------------------------ 9.4/39.4 MB 22.2 MB/s eta 0:00:02
   ----------- ---------------------------- 10.8/39.4 MB 23.4 MB/s eta 0:00:02
   ------------ --------------------------- 11.9/39.4 MB 23.4 MB/s eta 0:00:02
   ------------- -------------------------- 13.1/39.4 MB 25.2 MB/s eta

In [ ]:
# First, install dependencies in your local environment (run in terminal, not in the notebook):
# pip install transformers torch torchvision pillow scikit-learn opencv-python-headless

import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image, ImageEnhance
from transformers import AutoImageProcessor, AutoModelForImageClassification, pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Load the pretrained HF model and processor
model_name = "Anwarkh1/Skin_Cancer-Image_Classification"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)
classifier = pipeline("image-classification", model=model, feature_extractor=processor)

# 2. Define your preprocessing function
def preprocess_and_detect_mole(uploaded_file):
    image = Image.open(uploaded_file)
    image_rgb = np.array(image.convert('RGB'))
    gray_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray_image, 100, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    mask = np.zeros(gray_image.shape, dtype=np.uint8)
    cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cropped_image = image_rgb[y:y+h, x:x+w]
    cropped_image_pil = Image.fromarray(cropped_image)
    return cropped_image_pil


# 3. Load your metadata CSV
metadata = pd.read_csv("challenge-2016-test_metadata_2025-05-14.csv")


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Device set to use cpu


In [6]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   isic_id                 379 non-null    object 
 1   attribution             379 non-null    object 
 2   copyright_license       379 non-null    object 
 3   age_approx              320 non-null    float64
 4   anatom_site_general     299 non-null    object 
 5   anatom_site_special     5 non-null      object 
 6   benign_malignant        378 non-null    object 
 7   clin_size_long_diam_mm  202 non-null    float64
 8   concomitant_biopsy      379 non-null    bool   
 9   diagnosis_1             379 non-null    object 
 10  diagnosis_2             376 non-null    object 
 11  diagnosis_3             375 non-null    object 
 12  diagnosis_4             167 non-null    object 
 13  diagnosis_5             57 non-null     object 
 14  diagnosis_confirm_type  282 non-null    ob

Drop NaNs

In [42]:
# 0) Clean up the ground-truth column
#   - drop any rows where 'benign_malignant' is blank/NaN
#   - normalize everything to lowercase strings
metadata = metadata.dropna(subset=["benign_malignant"]).copy()
metadata["benign_malignant"] = metadata["benign_malignant"].astype(str).str.lower()

# (Optional) If you want to be extra safe, filter to only the two labels you expect:
metadata = metadata[
    metadata["benign_malignant"].isin(["benign", "malignant"])
].reset_index(drop=True)

Inspect prediction labels of model

In [43]:
# Inspect model’s label mapping
id2label = model.config.id2label
print("Model id2label:", id2label)
# e.g.: {0: 'benign', 1: 'malignant'}


Model id2label: {0: 'benign_keratosis-like_lesions', 1: 'basal_cell_carcinoma', 2: 'actinic_keratoses', 3: 'vascular_lesions', 4: 'melanocytic_Nevi', 5: 'melanoma', 6: 'dermatofibroma'}


## Test performance of model

In [52]:
# 1) Define a mapping from the model’s 7 classes to your 2 classes:
fine_to_binary = {
    "benign_keratosis-like_lesions": "benign",
    "basal_cell_carcinoma":          "malignant",
    "actinic_keratoses":             "benign",
    "vascular_lesions":              "benign",
    "melanocytic_Nevi":              "benign",
    "melanoma":                      "malignant",
    "dermatofibroma":                "benign",
}

# 2) Run your batch (or looped) inference to get `results`:
# (Either as a list-of-lists from pipeline or inside your loop.)
# Here’s the batch example again:
imgs, true_labels = [], []
for _, row in metadata.iterrows():
    path = os.path.join("ISIC-images", f"{row['isic_id']}.jpg")
    if not os.path.exists(path):
        continue
    imgs.append(preprocess_and_detect_mole(path))
    true_labels.append(row["benign_malignant"].lower())

results = classifier(imgs, batch_size=16)

# 3) Collapse the predicted labels:
pred_labels = [
    fine_to_binary[result[0]["label"]]  # e.g. "melanoma" → "malignant"
    for result in results
]

# 4) Now `true_labels` vs `pred_labels` are both in {"benign","malignant"}:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(true_labels, pred_labels))
print(classification_report(true_labels, pred_labels))
print(confusion_matrix(true_labels, pred_labels))


Accuracy: 0.7526595744680851
              precision    recall  f1-score   support

      benign       0.89      0.80      0.84       303
   malignant       0.40      0.58      0.47        73

    accuracy                           0.75       376
   macro avg       0.64      0.69      0.66       376
weighted avg       0.79      0.75      0.77       376

[[241  62]
 [ 31  42]]


In [16]:
confusion_matrix(true_labels, pred_labels)

array([[241,  62],
       [ 31,  42]], dtype=int64)